In [ ]:
# 데이터셋의 결측값을 다른 값으로 대체하기
# inplace = True 옵션으로 원본데이터 변경

df['CRIM'].fillna(df['CRIM'].mean(), inplace=True) # 평균값
df['INDUS'].fillna(df['INDUS'].mean(), inplace=True) # 평균값
df['AGE'].fillna(df['AGE'].mean(), inplace=True) # 평균값
df['LSTAT'].fillna(df['LSTAT'].mean(), inplace=True) # 평균값

# 그 외 결측값이 있는 행은 제거하기
df.dropna(inplace=True)

print('각 컬럼들의 결측값 현황')
print(df.isna().sum())  # 결측값이 잘 처리되었는지 확인 (결측값을 대체하거나 결측값이 있는 행을 제거해서 각 컬럼의 결측값이 0인 것을 알 수 있다.)
print('-'*130)
df.info() # 데이터프레임 정보 다시 확인 (총 506행의 데이터에서 466행의 결측값 없는 데이터로 바뀐 것을 확인 가능) (❗️ 결측값이 있는 행을 제거했을 뿐 인덱스는 기존과 같이 남아있다는 것 유의)

이 경우 모델 평가
Mean Squared Error: 9.709087415273192
R^2 Score: 0.7264160944964468

In [ ]:
# 결측값을 모두 평균값으로 대체한 케이스

df.fillna(df.mean(), inplace=True) # 평균값

이 경우 모델 평가
Mean Squared Error: 11.590991228748717
R^2 Score: 0.6562403686680377

In [ ]:
# 결측값을 모두 제거하고 학습한 케이스
# inplace = True 옵션으로 원본데이터 변경

# 결측값이 있는 행은 제거하기
df.dropna(inplace=True)

print('각 컬럼들의 결측값 현황')
print(df.isna().sum())  # 결측값이 잘 처리되었는지 확인 (결측값을 대체하거나 결측값이 있는 행을 제거해서 각 컬럼의 결측값이 0인 것을 알 수 있다.)
print('-'*130)
df.info() # 데이터프레임 정보 다시 확인 (총 506행의 데이터에서 466행의 결측값 없는 데이터로 바뀐 것을 확인 가능) (❗️ 결측값이 있는 행을 제거했을 뿐 인덱스는 기존과 같이 남아있다는 것 유의)

이 경우 모델 평가
Mean Squared Error: 12.393410369400286
R^2 Score: 0.5920913013754535

In [ ]:
# 결측값을 결측값을 예측하는 선형회귀모델로 채워준 케이스 (오히려 단순하게 평균값으로 대체하고 결측행을 제거한 것보다 성능이 떨어짐)

# column_with_na 열에 있는 결측값을 다른 열(feature1, feature2)을 사용하여 회귀 모델을 통해 예측하고, 그 예측된 값으로 결측값을 대체하는 과정
# 모델은 결측값이 있으면 학습할 수 없으니, 모든 입력 변수에서 결측값이 없는 데이터를 사용하는 것이 중요하다.

from sklearn.linear_model import LinearRegression

column_with_na = df.columns[df.isnull().any()]

for column in column_with_na:
    # 결측값이 있는 열과 없는 열 분리
    df_with_na = df[df[column].isnull()] # df_with_na는 해당 열에서 결측값이 있는 행들만 포함하는 데이터(df[불리언 시리즈]는 True인 행들을 선택하여 새로운 데이터프레임을 생성한다.)
    df_without_na = df[df[column].notnull()] #  df_without_na는 해당 열에서 결측값이 없는 행들만 포함하는 데이터프레임

    # 회귀 모델 학습
    model = LinearRegression()  
                    # 훈련용 입력 데이터         # 훈련용 타겟 데이터
    model.fit(df_without_na[['NOX', 'RM']], df_without_na[column])

    # 결측값 예측                         # 실제 테스트용 입력 데이터
    predicted_values = model.predict(df_with_na[['NOX', 'RM']])

    # 예측된 값으로 결측값 대체
    df.loc[df[column].isnull(), column] = predicted_values

이 경우 모델 평가
Mean Squared Error: 13.04010540136346
R^2 Score: 0.6132632889770075

선형회귀모델은 이쯤이 최선인 듯 함 ~